In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import MySQLdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [95]:
# # 날짜 dataframe 만들기
# start_date = pd.to_datetime('2015-01-01') ## 시작 날짜
# end_date = pd.to_datetime('2022-03-07') ## 마지막 날짜
 
# dates = pd.date_range(start_date,end_date,freq='M') ## 월단위로 생성
# dates = pd.DataFrame(dates, columns=['date'])

In [42]:
# 년,월,일 -> 날짜
def change_date(x):
    ret = []
    for da in x.split():
        ret.append(da[:-1])
    return '.'.join(ret)

In [150]:
# K, M, B 변환
def m_to_k(x):
    if x[-1] == 'M':
        return float(x[:-1]) * 1000
    elif x[-1] == 'K':
        return float(x[:-1])
    elif x[-1] == 'B':
        return float(x[:-1]) * 1000000
    else:
        return np.nan

In [ ]:
# 나스닥 종합지수, 상해 종합지수, 달러지수 병합

In [199]:
## 나스닥
df_nasdaq = pd.read_csv('나스닥종합지수.csv')
df_nasdaq.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_nasdaq.rename(columns={'날짜':'date', '종가':'nasdaq', '거래량':'nasdaq_거래량', '변동 %':'nasdaq_volatility'},
                 inplace=True)

### 날짜 전처리
df_nasdaq.date = df_nasdaq.date.apply(change_date)
df_nasdaq.date = pd.to_datetime(df_nasdaq.date)

### 가격 float타입
df_nasdaq.nasdaq = df_nasdaq.nasdaq.str.replace(',','').astype('float')

### 거래량 전처리
df_nasdaq['nasdaq_거래량'] = df_nasdaq['nasdaq_거래량'].apply(m_to_k)

### 변동% 전처리
df_nasdaq['nasdaq_volatility'] = df_nasdaq.nasdaq_volatility.apply(lambda x : float(x[:-1]))

In [198]:
## 상해 종합지수
df_ssec = pd.read_csv('상하이종합 내역.csv')
df_ssec.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_ssec.rename(columns={'날짜':'date', '종가':'ssec', '거래량':'ssec_거래량', '변동 %':'ssec_volatility'},
               inplace=True)

### 날짜 전처리
df_ssec.date = df_ssec.date.apply(change_date)
df_ssec.date = pd.to_datetime(df_ssec.date)

### 가격 float타입
df_ssec.ssec = df_ssec.ssec.str.replace(',','').astype('float')

### 거래량 전처리
df_ssec['ssec_거래량'] = df_ssec['ssec_거래량'].apply(m_to_k)

### 변동% 전처리
df_ssec['ssec_volatility'] = df_ssec.ssec_volatility.apply(lambda x : float(x[:-1]))

In [223]:
## 달러지수
df_dollar = pd.read_csv('달러 지수.csv')
df_dollar.drop(['오픈','고가','저가', '거래량'], axis=1, inplace=True)
df_dollar.rename(columns={'날짜':'date', '종가':'dollar', '변동 %':'dollar_volatility'}, inplace=True)

### 날짜 전처리
df_dollar.date = df_dollar.date.apply(change_date)
df_dollar.date = pd.to_datetime(df_dollar.date)

### 변동% 전처리
df_dollar['dollar_volatility'] = df_dollar.dollar_volatility.apply(lambda x : float(x[:-1]))

In [224]:
## 병합
df_composite_index = pd.merge(df_nasdaq, df_ssec, how='outer', on='date')
df_composite_index = pd.merge(df_composite_index, df_dollar, how='outer', on='date')
# df_composite_index.to_csv('종합지수_merge.csv')

In [ ]:
# 희토류, 리튬, 코발트 병합

In [209]:
## 리튬
df_li = pd.read_excel('리튬 가격.xlsx')
df_li.drop(['등락가', '재고량'], axis = 1, inplace=True)
df_li.columns = 'date', '리튬가격', '리튬변동률'

C:\Users\bitcamp\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [210]:
## 코발트
df_co = pd.read_excel('코발트 가격.xlsx')
df_co.drop(['등락가', '재고량'], axis = 1, inplace = True)
df_co.columns = 'date', '코발트가격', '코발트변동률'

In [211]:
## 희토류
df_ra = pd.read_excel('희토류 가격.xlsx')
df_ra.drop(['등락가', '재고량'], axis = 1, inplace = True)
df_ra.columns = 'date', '희토류가격', '희토류변동률'

In [212]:
## 병합
merge_lico = pd.merge(df_li, df_co, on = 'date', how = 'outer')
merge_licora = pd.merge(merge_lico, df_ra, on = 'date', how = 'outer')
# merge_licora.to_csv('배터리원자재_merge.csv')

In [ ]:
# 재무제표

In [399]:
## 재무제표 전처리
df_fn_statements = pd.read_csv('tesla 재무제표.csv')
df_fn_statements = df_fn_statements.set_index('Unnamed: 1')
df_fn_statements = df_fn_statements.drop(['Unnamed: 0','초이스스탁US 프리미엄', '프리미엄', '무료'], axis = 1)
df_fn_statements.dropna(inplace = True)
df_fn_statements = df_fn_statements.T
df_fn_statements = df_fn_statements.astype('object')
df_fn_statements = df_fn_statements.reset_index()
df_fn_statements = df_fn_statements.rename_axis(columns={'Unnamed: 1' : ''})
df_fn_statements = df_fn_statements.rename(columns={'index':'date'})
df_fn_statements.date = df_fn_statements.date.apply(lambda x : x.replace('.','/'))
df_fn_statements.date = df_fn_statements.date.apply(lambda x : '/'.join(x.split('/')[::-1]))
df_fn_statements.date = pd.to_datetime(df_fn_statements.date)
df_fn_statements.drop(['중단손익'], axis=1, inplace=True)

drop_col = ['매입채무등', '단기차입금', '장기차입금', '이연수익', '이연세부채','기타부채', '부채총계',
            '이익잉여금', '기타포괄익', '자본총계', '영업활동', '투자활동', '재무활동', '환율변동',
            '현금의증감', '자본적지출', '잉여현금', '현금성자산', '단기투자', '매출채권등', '재고자산',
            '장기투자', '유형자산', '무형자산', '이연세자산', '기타자산','EBITDA', '법인세', '판매관리비',
            '연구개발비']

df_fn_statements.drop(drop_col, axis=1, inplace=True)
df_fn_statements = df_fn_statements.iloc[:28]

# float 타입변환
col = df_fn_statements.columns.values
for c in col[1:]:
    df_fn_statements.loc[:,c] = df_fn_statements.loc[:,c].apply(lambda x : x.replace(',',''))
    df_fn_statements[c] = df_fn_statements[c].astype('float')

# 이익률 계산
df_fn_statements['매출총이익률'] = (df_fn_statements['매출총이익']/df_fn_statements['매출액'])*100
df_fn_statements['영업이익률'] = (df_fn_statements['영업이익']/df_fn_statements['매출액'])*100
# df_fn_statements.to_csv('fn_statements.csv')

In [354]:
# 리튬관련 etf

In [356]:
## LIT etf
df_lit = pd.read_csv('리튬 etf LIT.csv')
df_lit.drop(['Open', 'High', 'Low', 'Close'], axis = 1, inplace=True)
df_lit.columns = 'date', 'LIT조정종가', 'LIT거래량'
df_lit['date'] = pd.to_datetime(df_lit['date'])

In [363]:
## SGL ETF
df_sgl = pd.read_csv('리튬전지 SGL etf.csv')
df_sgl.drop(['오픈', '고가', '저가', '거래량'], axis = 1, inplace = True)
df_sgl.columns = 'date', 'SGL종가', 'SGL변동률'
df_sgl['date'] = df_sgl['date'].apply(change_date)
df_sgl['date'] = pd.to_datetime(df_sgl['date'])

In [372]:
## 리튬 etf 병합
df_lithium_etf = pd.merge(df_lit, df_sgl, on = 'date', how = 'outer')
# df_lithium_etf.to_csv('lithium_etf_merge')

In [400]:
# 반도체관련 etf

In [407]:
## psi etf (미국)
df_psi = pd.read_csv("미국 반도체 etf PSI.csv")
df_psi = df_psi.rename(columns={'날짜':'date'})
df_psi = df_psi.rename(columns={'거래량':'PSI거래량','종가':'PSI종가'})
df_psi.drop(['오픈','고가','저가','변동 %'], axis=1, inplace=True)
df_psi.date = pd.to_datetime(df_psi.date.apply(change_date))
df_psi.PSI거래량 = df_psi.PSI거래량.apply(m_to_k).astype('float')

In [408]:
## soxx etf (미국)
df_soxx = pd.read_csv("미국 반도체 etf SOXX.csv")
df_soxx = df_soxx.rename(columns = {'날짜' : 'date','거래량' : 'SOXX거래량','종가' : 'SOXX종가'})
df_soxx.drop(['오픈','고가','저가','변동 %'], axis=1, inplace=True)
df_soxx.date = pd.to_datetime(df_soxx.date.apply(change_date))
df_soxx.SOXX거래량 = df_soxx.SOXX거래량.apply(m_to_k).astype("float")

In [409]:
## soxl etf (미국)
df_soxl = pd.read_csv("미국 반도체 etf SOXL.csv")
df_soxl = df_soxl.rename(columns = {'날짜' : 'date','거래량' : 'SOXL거래량','종가' : 'SOXL종가'})
df_soxl.drop(['오픈','고가','저가','변동 %'], axis=1, inplace=True)
df_soxl.date = pd.to_datetime(df_soxl.date.apply(change_date))
df_soxl.SOXL거래량 = df_soxl.SOXL거래량.apply(m_to_k).astype("float")

In [413]:
## 반도체 etf merge
merge_outer = pd.merge(df_psi, df_soxx,how='outer', on='date')
merge_outer2 = pd.merge(merge_outer,df_soxl, how='outer', on='date')
# merge_outer2.to_csv('반도체 etf merge.csv')

In [ ]:
# 전기차관련 etf

In [414]:
## driv etf (미국)
df_driv = pd.read_csv("미국 전기차 erf DRIV.csv")
df_driv = df_driv.rename(columns = {'날짜':'date', '종가':'DRIV종가','거래량':'DRIV거래량'})
df_driv.drop(['오픈','고가','저가','변동 %'], axis =1, inplace=True)
df_driv.date = pd.to_datetime(df_driv.date.apply(change_date))
df_driv.DRIV거래량 = df_driv.DRIV거래량.apply(m_to_k).astype('float')

In [415]:
## acdc etf (캐나다)
df_acdc = pd.read_csv("캐나다 전기차 etf ACDC.csv")
df_acdc = df_acdc.rename(columns = {'날짜':'date', '종가':'ACDC종가','거래량':'ACDC거래량'})
df_acdc.drop(['오픈','고가','저가','변동 %'], axis =1, inplace=True)
df_acdc.date = pd.to_datetime(df_acdc.date.apply(change_date))
df_acdc.ACDC거래량 = df_acdc.ACDC거래량.apply(m_to_k).astype('float')

In [418]:
## 전기차 etf 병합
outer_merge = pd.merge(df_driv, df_acdc, how='outer', on='date')
outer_merge.to_csv('전기차 etf merge.csv')

In [419]:
# 테슬라 주가(target)

In [421]:
## 테슬라
df_tesla = pd.read_csv('테슬라 주가.csv')
df_tesla.drop(['오픈','고가','저가'], axis=1, inplace=True)
df_tesla.rename(columns={'날짜':'date', '종가':'tesla', '거래량':'tesla_거래량', '변동 %':'tesla_volatility'},
                 inplace=True)

### 날짜 전처리
df_tesla.date = df_tesla.date.apply(change_date)
df_tesla.date = pd.to_datetime(df_tesla.date)

### 거래량 전처리
df_tesla['tesla_거래량'] = df_tesla['tesla_거래량'].apply(m_to_k)

### 변동% 전처리
df_tesla['tesla_volatility'] = df_tesla.tesla_volatility.apply(lambda x : float(x[:-1]))

In [ ]:
# 유가, 유가 선물지수, 탄소배출권

In [438]:
df_oli_price = pd.read_csv('국제 유가.csv', skiprows=4)

In [439]:
df_oli_price

,Day,Cushing OK Crude Oil Future Contract 1 Dollars per Barrel
0,03/1/2022,103.41
1,02/28/2022,95.72
2,02/25/2022,91.59
3,02/24/2022,92.81
4,02/23/2022,92.10
...,...,...
9765,04/8/1983,30.38
9766,04/7/1983,30.17
9767,04/6/1983,29.92
9768,04/5/1983,29.71
